In [4]:
from neuroposelib import read
from neuroposelib import vis
import numpy as np
import time
from IPython.display import Video
from pathlib import Path
import matplotlib.pyplot as plt
# %matplotlib inline
from neuroposelib import preprocess
from neuroposelib import write
from neuroposelib import features
from neuroposelib import DataStruct as ds
from neuroposelib.embed import Embed
from neuroposelib import analysis
import pandas as pd
from neuroposelib.embed import Watershed

config = read.config('/home/lq53/mir_repos/BBOP/random tests/dappy_backups/iter_com_features/mir_1.yaml')

connectivity = read.connectivity(
    path=config["skeleton_path"], skeleton_name=config["skeleton_name"]
)

# Make out_path
Path(config["out_path"]).mkdir(parents=True, exist_ok=True)

# meta, meta_by_frame = read.meta(config["data_path"] + "demo_meta.csv", id=ids)
pose, ids, meta, meta_by_frame = read.pose_from_meta(
    path=config["meta_path"], connectivity=connectivity, key="Prediction_path", file_type="dannce"
)

Path(config["out_path"]).mkdir(parents=True, exist_ok=True)

# write.pose_h5(pose,ids, config['out_path'] + 'pose_merged_newcol.h5')


pose = preprocess.rotate_spine(preprocess.center_spine(pose, keypt_idx=4), keypt_idx=[4, 3])


115it [00:06, 17.51it/s]


Centering poses to mid spine ...
Rotating spine to xz plane ... 


In [9]:
from tqdm import tqdm
from typing import Optional, Union, List, Tuple, Type

def pca(
    features: np.ndarray,
    labels: List,
    categories: List[str] = ["vel", "ego_euc", "ang", "avel"],
    n_pcs: int = 10,
    downsample: int = 1,
    method="fbpca",
):
    print("Calculating principal components ... ")

    # Initializing the PCA method
    # if method.startswith("torch"):
    #     import torch

    #     pca_feats = torch.zeros(features.shape[0], len(categories) * n_pcs)
    #     features = torch.tensor(features)
    # else:
    # Centering the features if not torch (pytorch does it itself)
    features = features - features.mean(axis=0)
    pca_feats = np.zeros(
        (features.shape[0], len(categories) * n_pcs), dtype=features.dtype
    )

    if method == "ipca":
        from sklearn.decomposition import IncrementalPCA

        pca = IncrementalPCA(n_components=n_pcs, batch_size=None)
    elif method.startswith("fbpca"):
        import fbpca

    num_cols = 0
    for i, cat in enumerate(tqdm(categories)):  # Iterate through each feature category
        cat += "_"
    
        cols_idx = [
            i
            for i, col in enumerate(labels)
            if (col.startswith(cat) or ("_" + cat in col))
            # if (col.decode('utf-8').startswith(cat) or ("_" + cat in col.decode('utf-8')))

        ]
        num_cols += len(cols_idx)
        # print(f"Category: {cat}, Matching labels: {[label for label in combined_labels if cat in label]}")
        if not cols_idx:
            print(f"Skipping category {cat}: no matching columns found.")
            continue

        if method == "ipca" or method == "sklearn_pca":
            # import pdb; pdb.set_trace()
            pca_feats[:, i * n_pcs : (i + 1) * n_pcs] = pca.fit_transform(
                features[:, cols_idx]
            )

        # elif method.startswith("torch"):
        #     feat_cat = features[:, cols_idx]
        #     if method.endswith("_gpu"):
        #         feat_cat = feat_cat.cuda()

        #     if "pca" in method:
        #         (_, _, V) = torch.pca_lowrank(feat_cat)
        #     elif "svd" in method:
        #         feat_cat -= feat_cat.mean()
        #         (_, _, V) = torch.linalg.svd(feat_cat)

        #     if method.endswith("_gpu"):
        #         pca_feats[:, i * n_pcs : (i + 1) * n_pcs] = (
        #             torch.matmul(feat_cat, V[:, :n_pcs]).detach().cpu()
        #         )
        #         feat_cat.detach().cpu()
        #         V.detach().cpu()
        #     else:
        #         pca_feats[:, i * n_pcs : (i + 1) * n_pcs] = torch.matmul(
        #             feat_cat, V[:, :n_pcs]
        #         )

        elif method == "fbpca":
            print(f"Category: {cat}, Columns: {cols_idx}, Feature shape: {features[:, cols_idx].shape}")

            (_, _, V) = fbpca.pca(
                features[::downsample, cols_idx].astype(np.float64), k=n_pcs
            )
            pca_feats[:, i * n_pcs : (i + 1) * n_pcs] = np.matmul(
                features[:, cols_idx], V.astype(features.dtype).T
            )

    # if method.startswith("torch_pca"):
    #     pca_feats = pca_feats.numpy()

    # assert num_cols == features.shape[1]

    pc_labels = [
        "_".join([cat, "pc" + str(i)]) for cat in categories for i in range(n_pcs)
    ]

    return pca_feats, pc_labels

In [ ]:
rel_vel, rel_vel_labels = features.get_velocities(
    pose,
    ids,
    connectivity.joint_names,
    joints=np.delete(np.arange(18), 4),
    widths=[5, 11, 51],
    f_s = 30
)

angles, angle_labels = features.get_angles(pose, connectivity.angles)

ego_pose, labels = features.get_ego_pose(pose, connectivity.joint_names)

# Compute and save Euler angles (if applicable)
euler_angles, euler_labels = features.get_euler_angles(pose, connectivity.angles)
# write.features_h5(
#     euler_angles, euler_labels, path=config["out_path"] + "euler_angles.h5"
# )

# Compute and save head angular velocities
head_angular = features.get_head_angular(pose, ids, widths=[5, 10, 50])
# write.features_h5(
#     head_angular, ["head_angular"], path=config["out_path"] + "head_angular.h5"
# )

# Angular Velocities
angular_vel, angular_vel_labels = features.get_angular_vel(
        angles,
        angle_labels,
        ids,
        widths=[5, 11, 51],
    )
# write.features_h5(
#     angular_vel, angular_vel_labels, path=config["out_path"] + "angular_velocity.h5"
# )
# # Ensure all feature datasets have the same number of samples
assert rel_vel.shape[0] == angles.shape[0] == ego_pose.shape[0]

# Combine features
combined_features = np.hstack([rel_vel, angles, ego_pose])

# Concatenate labels
combined_labels = rel_vel_labels + angle_labels + labels



# Perform PCA on the combined features
# pca_features, pca_labels = pca(
#     features=combined_features,
#     labels=combined_labels,
#     categories=["vel", "ang", "ego_pose"],
#     n_pcs=10,
#     downsample=1,
#     method="fbpca",
# )

# write.features_h5(pca_features, pca_labels, path=config["out_path"] + "_combined_pca.h5")


# pc_feats, pc_labels = pca_features, pca_labels

# wlet_feats, wlet_labels = features.wavelet(
#     pc_feats, pc_labels, ids, f_s=30, freq=np.linspace(1, 25, 25), w0=5
# )

# write.features_h5(wlet_feats, wlet_labels, path=config["out_path"] + "_wavelets.h5")


In [17]:
# somehow all comb looks very similar, cannot tell whether there is a bug so it's trash or something
import os

# Define combinations of features and labels
# combinations = [
#     {
#         "name": "vel_ang_ego",  # Combination name
#         "features": [rel_vel, angles, ego_pose],
#         "labels": rel_vel_labels + angle_labels + labels,
#         "categories": ["vel", "ang", "ego_pose"],
#     },
#     {
#         "name": "vel_ang",  # Another combination
#         "features": [rel_vel, angles],
#         "labels": rel_vel_labels + angle_labels,
#         "categories": ["vel", "ang"],
#     },
#     # Add more combinations as needed
# ]
# Define combinations of features and labels
combinations = [
    
    # {
    #     "name": "all_features",  # All features combined
    #     "features": [rel_vel, angles, ego_pose],
    #     "labels": rel_vel_labels + angle_labels + labels,
    #     "categories": ["vel", "ang", "ego_pose"],
    # },
    # {
    #     "name": "vel_angvel_ang",  # Velocity, angle velocity, and angle
    #     "features": [rel_vel, angles],  # Assuming angle velocity is part of angles or needs another array
    #     "labels": rel_vel_labels + angle_labels,  # Update if angle velocity labels are separate
    #     "categories": ["vel", "ang"],
    # },
    # # Existing combinations
    # {
    #     "name": "vel_ang_ego",
    #     "features": [rel_vel, angles, ego_pose],
    #     "labels": rel_vel_labels + angle_labels + labels,
    #     "categories": ["vel", "ang", "ego_pose"],
    # },
    # {
    #     "name": "vel_ang",
    #     "features": [rel_vel, angles],
    #     "labels": rel_vel_labels + angle_labels,
    #     "categories": ["vel", "ang"],
    # },
]

for comb in combinations:
    comb_name = comb["name"]
    combined_features = np.hstack(comb["features"])
    combined_labels = comb["labels"]
    categories = comb["categories"]

    # Ensure output directory for the combination exists
    comb_dir = f"{config['out_path']}/{comb_name}"
    os.makedirs(comb_dir, exist_ok=True)

    # PCA on combined features
    pca_features, pca_labels = pca(
        features=combined_features,
        labels=combined_labels,
        categories=categories,
        n_pcs=10,
        downsample=1,
        method="fbpca",
    )

    # Save PCA results
    pca_path = f"{comb_dir}/_combined_pca.h5"
    write.features_h5(pca_features, pca_labels, path=pca_path)

    # Wavelet transformation
    wlet_feats, wlet_labels = features.wavelet(
        pca_features, pca_labels, ids, f_s=30, freq=np.linspace(1, 25, 25), w0=5
    )
    wlet_path = f"{comb_dir}/_wavelets.h5"
    write.features_h5(wlet_feats, wlet_labels, path=wlet_path)

    # PCA on wavelet features
    pc_wlet, pc_wlet_labels = pca(
        wlet_feats,
        wlet_labels,
        categories=[f"wlet_{cat}" for cat in categories],
        n_pcs=5,
        method="fbpca",
    )

    del wlet_feats, wlet_labels
    pc_feats = np.hstack((pca_features, pc_wlet))
    pc_labels = combined_labels + pc_wlet_labels
    del pc_wlet, pc_wlet_labels

    # Save final PCA results
    final_pca_path = f"{comb_dir}/_pca_on_wavelets.h5"
    write.features_h5(pc_feats, pc_labels, path=final_pca_path)

    # Data structure and embedding
    data_obj = ds.DataStruct(
        pose=pose,
        id=ids,
        meta=meta,
        meta_by_frame=meta_by_frame,
        connectivity=connectivity,
    )
    data_obj.features = pc_feats
    data_obj = data_obj[:: config["downsample"], :]
    embedder = Embed(
        embed_method=config["single_embed"]["method"],
        perplexity=config["single_embed"]["perplexity"],
        lr=config["single_embed"]["lr"],
    )
    data_obj.embed_vals = embedder.embed(data_obj.features, save_self=True)

    # Watershed clustering
    data_obj.ws = Watershed(
        sigma=config["single_embed"]["sigma"], max_clip=1, log_out=True, pad_factor=0.05
    )
    data_obj.data["Cluster"] = data_obj.ws.fit_predict(data=data_obj.embed_vals)

    # Save data object
    data_pickle_path = f"{comb_dir}/data_obj.pkl"
    data_obj.write_pickle(data_pickle_path)

    # Save cluster frequency
    freq, combined_keys = analysis.cluster_freq_by_cat(
        data_obj.data["Cluster"].values, cat=data_obj.id
    )
    freq_df = pd.DataFrame(freq.T, columns=combined_keys)
    freq_csv_path = f"{comb_dir}/cluster_occupancy.csv"
    freq_df.to_csv(freq_csv_path)

    # Visualization
    vis.pose.sample_grid3D(
        pose - pose.mean(axis=-2, keepdims=True),
        connectivity=connectivity,
        labels=data_obj.data["Cluster"],
        n_samples=9,
        centered=True,
        N_FRAMES=100,
        fps=30,
        dpi=100,
        watershed=data_obj.ws,
        embed_vals=None,
        VID_NAME=f"{comb_name}_cluster",
        filepath=comb_dir,
    )


Calculating principal components ... 


  0%|          | 0/3 [00:00<?, ?it/s]

Category: vel_, Columns: [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29, 30, 31, 32, 33, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 46, 47, 48, 49, 50, 51, 52, 53, 54, 55, 56, 57, 58, 59, 60, 61, 62, 63, 64, 65, 66, 67, 68, 69, 70, 71, 72, 73, 74, 75, 76, 77, 78, 79, 80, 81, 82, 83, 84, 85, 86, 87, 88, 89, 90, 91, 92, 93, 94, 95, 96, 97, 98, 99, 100, 101, 102, 103, 104, 105, 106, 107, 108, 109, 110, 111, 112, 113, 114, 115, 116, 117, 118, 119, 120, 121, 122, 123, 124, 125, 126, 127, 128, 129, 130, 131, 132, 133, 134, 135, 136, 137, 138, 139, 140, 141, 142, 143, 144, 145, 146, 147, 148, 149, 150, 151, 152, 153, 154, 155, 156, 157, 158, 159, 160, 161, 162, 163, 164, 165, 166, 167, 168, 169, 170, 171, 172, 173, 174, 175, 176, 177, 178, 179, 180, 181, 182, 183, 184, 185, 186, 187, 188, 189, 190, 191, 192, 193, 194, 195, 196, 197, 198, 199, 200, 201, 202, 203, 204, 205, 206, 207, 208, 209, 210, 211, 212, 213, 214, 215, 216,

 33%|███▎      | 1/3 [00:28<00:56, 28.02s/it]

Category: ang_, Columns: [408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443, 444, 445, 446, 447, 448, 449, 450, 451, 452, 453, 454, 455, 456, 457, 458, 459, 460, 461, 462], Feature shape: (3539000, 55)


...

 98%|█████████▊| 101/103 [54:56<01:06, 33.24s/it]

[447840, 1299430, 1878880, 441350, 2526920, 1251610, 1257680, 459260, 96950]


 99%|█████████▉| 102/103 [55:29<00:33, 33.17s/it]

[1603410, 2804740, 2368460, 1090490, 1090760, 2440380, 97900, 2161930, 483190]


100%|██████████| 103/103 [56:02<00:00, 32.65s/it]


In [12]:


# # PCA on wavelet features
# pc_wlet, pc_wlet_labels = pca(
#     wlet_feats,
#     wlet_labels,
#     # categories=["wlet_ego_euc"],
#     categories=["wlet_ang"],
#     n_pcs=5,
#     method="fbpca",
# )

# del wlet_feats, wlet_labels
# pc_feats = np.hstack((pc_feats, pc_wlet))
# pc_labels += pc_wlet_labels
# del pc_wlet, pc_wlet_labels

# write.features_h5(
#     pc_feats, pc_labels, path="".join([config["out_path"], "_pca_on_wavelets.h5"])
# )




# data_obj = ds.DataStruct(
#     pose=pose,
#     id=ids,
#     meta=meta,
#     meta_by_frame=meta_by_frame,
#     connectivity=connectivity,
# )

# data_obj.features = pc_feats
# # When using high framerate data, downsampling may be necessary in order to 
# # discover granular structure in embedding
# data_obj = data_obj[:: config["downsample"], :]


# embedder = Embed(
#     embed_method=config["single_embed"]["method"],
#     perplexity=config["single_embed"]["perplexity"],
#     lr=config["single_embed"]["lr"],
# )
# data_obj.embed_vals = embedder.embed(data_obj.features, save_self=True)

# # Watershed clustering
# data_obj.ws = Watershed(
#     sigma=config["single_embed"]["sigma"], max_clip=1, log_out=True, pad_factor=0.05
# )
# data_obj.data["Cluster"] = data_obj.ws.fit_predict(data=data_obj.embed_vals)
# print("Writing Data Object to pickle")
# data_obj.write_pickle(''.join([config['out_path'],'/']))



# freq, combined_keys = analysis.cluster_freq_by_cat(
#                         data_obj.data["Cluster"].values, cat=data_obj.id
#                     )
# freq_df = pd.DataFrame(freq.T, columns=combined_keys)
# freq_df.to_csv(''.join([config['out_path'],'/cluster_occupancy.csv']))

# vis.pose.sample_grid3D(
#     pose-pose.mean(axis=-2, keepdims=True),
#     connectivity=connectivity,
    
#     labels=data_obj.data["Cluster"],
#     n_samples=9,
#     centered=True,
#     N_FRAMES=100,
#     fps=30,
#     dpi=100,
#     watershed=data_obj.ws,
#     embed_vals=None,
#     VID_NAME = "cluster",
#     filepath=config["out_path"],
# )


Calculating principal components ... 


  0%|          | 0/1 [00:00<?, ?it/s]

Category: wlet_ang_, Columns: [250, 251, 252, 253, 254, 255, 256, 257, 258, 259, 260, 261, 262, 263, 264, 265, 266, 267, 268, 269, 270, 271, 272, 273, 274, 275, 276, 277, 278, 279, 280, 281, 282, 283, 284, 285, 286, 287, 288, 289, 290, 291, 292, 293, 294, 295, 296, 297, 298, 299, 300, 301, 302, 303, 304, 305, 306, 307, 308, 309, 310, 311, 312, 313, 314, 315, 316, 317, 318, 319, 320, 321, 322, 323, 324, 325, 326, 327, 328, 329, 330, 331, 332, 333, 334, 335, 336, 337, 338, 339, 340, 341, 342, 343, 344, 345, 346, 347, 348, 349, 350, 351, 352, 353, 354, 355, 356, 357, 358, 359, 360, 361, 362, 363, 364, 365, 366, 367, 368, 369, 370, 371, 372, 373, 374, 375, 376, 377, 378, 379, 380, 381, 382, 383, 384, 385, 386, 387, 388, 389, 390, 391, 392, 393, 394, 395, 396, 397, 398, 399, 400, 401, 402, 403, 404, 405, 406, 407, 408, 409, 410, 411, 412, 413, 414, 415, 416, 417, 418, 419, 420, 421, 422, 423, 424, 425, 426, 427, 428, 429, 430, 431, 432, 433, 434, 435, 436, 437, 438, 439, 440, 441, 442, 443,

100%|██████████| 1/1 [00:17<00:00, 17.25s/it]


Running fitsne via openTSNE
--------------------------------------------------------------------------------
TSNE(early_exaggeration=12, exaggeration=1.5, n_jobs=-1,
     negative_gradient_method='fft', neighbors='annoy', perplexity=50,
     verbose=True)
--------------------------------------------------------------------------------
===> Finding 150 nearest neighbors using Annoy approximate search using euclidean distance...
   --> Time elapsed: 67.33 seconds
===> Calculating affinity matrix...
   --> Time elapsed: 10.09 seconds
===> Calculating PCA-based initialization...
   --> Time elapsed: 0.05 seconds
===> Running optimization with exaggeration=12.00, lr=29491.67 for 250 iterations...
Iteration   50, KL divergence 7.1776, 50 iterations in 3.4891 sec
Iteration  100, KL divergence 7.1621, 50 iterations in 3.5430 sec
Iteration  150, KL divergence 7.1514, 50 iterations in 3.4776 sec
Iteration  200, KL divergence 7.1488, 50 iterations in 3.5541 sec
Iteration  250, KL divergence 7.147

...

 98%|█████████▊| 103/105 [54:58<01:03, 31.67s/it]

[1004600, 1139020]


 99%|█████████▉| 104/105 [55:07<00:24, 24.84s/it]

[2931990, 2593810, 1123870, 2740100, 1124110, 936370, 1615090, 1178710, 3477790]


100%|██████████| 105/105 [55:40<00:00, 31.81s/it]
